In [ ]:
! [ -d "PetImages" ] && echo "skipping" || (wget -nc --no-check-certificate https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip && unzip kagglecatsanddogs_3367a.zip -d .)

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
from sap_computer_vision.datasets import image_folder as imgf
import pathlib, json


dataset_folder = pathlib.Path('PetImages/')
datasets, class_names = imgf.split_and_register('dogs_vs_cats',
                                                base_dir=dataset_folder,
                                                splits={'test': 0.1, 'validation': 0.1, 'train': 0.8})

In [ ]:
from sap_computer_vision import setup_loggers, get_cfg, get_config_file

cfg = get_cfg()
cfg.merge_from_file(get_config_file('Base-EarlyStopping'))
cfg.merge_from_file(get_config_file('Base-Evaluation'))
cfg.merge_from_file(get_config_file('ImageClassifier/Resnet50'))

cfg.OUTPUT_dir = 'model_out'
cfg.DATASETS.TRAIN = ('dogs_vs_cats_train', )
cfg.DATASETS.TEST = ('dogs_vs_cats_validation', )

cfg.SOLVER.MAX_ITER = 2000
cfg.SOLVER.BASE_LR = 0.01
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.IMS_PER_BATCH_EVAL = 32

In [ ]:
from sap_computer_vision.engine import ImageClassificationTrainer

In [ ]:
trainer = ImageClassificationTrainer(cfg)
trainer.resume_or_load(resume=False)

In [ ]:
trainer.train()

In [ ]:
cfg.DATASETS.TEST = ('dogs_vs_cats_test', )
metrics = trainer.test(cfg, trainer.model)
print(json.dumps(metrics, ident=2))